## Load common-code
Load code that enables GIT access and grabs all category names


In [82]:
%run ./common-code.ipynb

[ContentFile(path="D2-documentation"), ContentFile(path="DATA"), ContentFile(path="README.md"), ContentFile(path="scripts")]
ContentFile(path="D2-documentation")
ContentFile(path="DATA")
ContentFile(path="README.md")
ContentFile(path="scripts")
https://raw.githubusercontent.com/ehsintegration/yfd-phd-bls-data/master/DATA/FATAL_all.csv
https://raw.githubusercontent.com/ehsintegration/yfd-phd-bls-data/master/DATA/NF_AGE_14to15_all.csv
https://raw.githubusercontent.com/ehsintegration/yfd-phd-bls-data/master/DATA/NF_AGE_16to19_all.csv
https://raw.githubusercontent.com/ehsintegration/yfd-phd-bls-data/master/DATA/NF_AGE_20to24_all.csv
https://raw.githubusercontent.com/ehsintegration/yfd-phd-bls-data/master/DATA/NF_AGE_25to34_all.csv
https://raw.githubusercontent.com/ehsintegration/yfd-phd-bls-data/master/DATA/NF_AGE_35to44_all.csv
https://raw.githubusercontent.com/ehsintegration/yfd-phd-bls-data/master/DATA/NF_AGE_45to54_all.csv
https://raw.githubusercontent.com/ehsintegration/yfd-phd-bls-da

## Create dict of SOC category names


In [83]:
soc_names = {}

#print(soc_array)
for elem in soc_array:
    if (elem.find("-")>=0):
        line  = elem.replace("-","")
        #print(line)
        key   = line.split(" ",1)[0]
        value = line.split(" ",1)[1].lstrip().rstrip()    
        soc_names.setdefault(key, []).append(value)

#index = 0
#for key in soc_names:
#    print(index, key, soc_names[key])
#    index = index + 1

## Select Generic Database to work with

In [84]:
fatalDB = []

desired_file = "FATAL_all.csv"

## Create FATAL dictionary

In [85]:
# pull in the data
# Iterate through all of the URLs
for index in contents:
    url = index.download_url
    
    # Is content_file a file with the .csv extension
    if (desired_file in url):
        print(url)
        
        # Grab the source contents of the file
        source = urllib.request.urlopen(url).read()
        
        #print(source)
        decoded_data = source.decode("utf-8")
        fatalDB.append(decoded_data)   

https://raw.githubusercontent.com/ehsintegration/yfd-phd-bls-data/master/DATA/FATAL_all.csv


In [86]:
# Create a dictionary of soc_codes
#
fatalDB_dict = {}

line = fatalDB[0].replace("\"","")
fatalDB_array= line.split("\n")

#print(fatalDB_array)

# Create Dictionary of SOC codes
#
for line in fatalDB_array[12:]:
    line_array = line.split(",")
    #key  = line_array[0].    [0:7].replace("-","")
    key  = line_array[0][6:12]
    data = line_array[1:]
    #print(line_array)
    
    # Replace empty values with zero
    if (len(key) > 1):
        #print(key,data[0:8])
        for index in range(0,8):
            if (len(data[index]) == 0):
                data[index] = '0'
        #print(key,data[0:8])
        
    # Convert to Integers
    if (len(key) > 1):
        #print(key,data[0:8])
        for index in range(0,8):
            data[index] = int(data[index])
        #print(key,data[0:8])   
            
    # Create dictionary entry
    if (len(key) > 1):
        fatalDB_dict.setdefault(key, []).append(data[0:8])
    
#index = 0
#for key in fatalDB_dict:
#    print(index, key, fatalDB_dict[key])
#    index = index + 1

In [87]:
print(fatalDB_dict['000000'][0])

[4693, 4628, 4585, 4821, 4836, 5190, 5147, 5250]


## Create SOC level Groups
Create SOC Level Groups for SOC work groups as a dictionary of keys

Level 1 "11XXXX", "12XXXX"....<br>
Level 2 "110XXX", "111XXX", "120XXX"...<br>
Level 3 "1101XX", "1102XX", "1201XX"......<br>

In [88]:
level1_sockeys = {}
level2_sockeys = {}
level3_sockeys = {}

for key in soc_dict.keys():
    lev1 = key[0:2]
    lev2 = key[0:3]
    lev3 = key[0:6]
    #print(lev1)
    level1_sockeys.setdefault(lev1, []).append(key)
    level2_sockeys.setdefault(lev2, []).append(key)
    level3_sockeys.setdefault(lev3, []).append(key)
    
#print(level1_sockeys)

## Create a count of DATABASE data for the diferent level soc groups

In [89]:
no_fatalDB_data_dict = {}

for index in level1_sockeys.keys():
    # running fatal_totals for 2011 to 2018
    if (len(index) > 0):
        fatal_totals = np.array([0,0,0,0,0,0,0,0])
        for key in level1_sockeys[index]:
            if key in fatalDB_dict.keys():
                #print(fatalDB_dict[key][0])
                fatal_totals = fatal_totals + np.array(fatalDB_dict[key][0])
                #print(key,fatalDB_dict[key])
            else:
                no_fatalDB_data_dict.setdefault(key, []).append(0)
                #print("No fatalDB data for ", key)
        category = soc_names[str(index)+"0000"]
        #print(index,category,fatal_totals)
        
print("") 

for index in level2_sockeys.keys():
    # running fatal_totals for 2011 to 2018
    if (len(index) > 0):
        fatal_totals = np.array([0,0,0,0,0,0,0,0])
        for key in level2_sockeys[index]:        
            if key in fatalDB_dict.keys():
                #print(fatalDB_dict[key][0])
                fatal_totals = fatal_totals + np.array(fatalDB_dict[key][0])
                #print(key,fatalDB_dict[key])
            else:
                no_fatalDB_data_dict.setdefault(key, []).append(0)
                #print("No fatalDB data for ", key)
        #category = soc_names[str(index)+"000"]
        #print(index,fatal_totals)
        
print("") 

for index in level3_sockeys.keys():
    # running fatal_totals for 2011 to 2018
    if (len(index) > 0):
        fatal_totals = np.array([0,0,0,0,0,0,0,0])
        for key in level3_sockeys[index]:
            if key in fatalDB_dict.keys():
                #print(fatalDB_dict[key][0])
                fatal_totals = fatal_totals + np.array(fatalDB_dict[key][0])
                #print(key,fatalDB_dict[key])
            else:
                no_fatalDB_data_dict.setdefault(key, []).append(0)
                #print("No fatalDB data for ", key)
        category = soc_names[str(index)]
        #print(index,category,fatal_totals)
        
#print(no_fatalDB_data_dict.keys())